In [22]:
import requests
from bs4 import BeautifulSoup

import pandas as pd

path = r'/Users/dewiindahpurnama_/Documents/Python/property-data/Sellers_Accounts_2025_12_27.xlsx'
df = pd.read_excel(path, dtype=str, sheet_name='Sheet0', usecols=['Record Id', 'Seller Account Name', 'Phone', 'Link'])



In [23]:
df.loc[df['Link'].notna()]

,Record Id,Seller Account Name,Phone,Link
2,zcrm_6965175000003577206,Victor Remy,NaN,https://matapropertybali.com/property/modern-4...
3,zcrm_6965175000003577207,Cedric Henry Levi,NaN,https://matapropertybali.com/property/modern-3...
4,zcrm_6965175000003577208,BSH,NaN,https://matapropertybali.com/property/villa-wi...
5,zcrm_6965175000003577209,Bali Tropical Sharing,NaN,https://matapropertybali.com/property/5-bedroo...
6,zcrm_6965175000003577210,BSH,NaN,https://matapropertybali.com/property/luxuriou...
...,...,...,...,...
527,zcrm_6965175000003616239,CP: Juli Optic,+62 831-1616-7305,https://matapropertybali.com/property/a-breath...
528,zcrm_6965175000003616240,Balix Development,NaN,https://matapropertybali.com/property/stylish-...
529,zcrm_6965175000003616241,CP: Lanny,+62 813-1920-0900,https://matapropertybali.com/property/discover...
530,zcrm_6965175000003616242,JSP Development,+62 812-3661-9575,https://matapropertybali.com/property/newly-bu...


In [ ]:
def get_property_data(url):
    if pd.isna(url):
        return {"Scraped_Title": "", "Scraped_Price": ""}

    soup = BeautifulSoup(requests.get(url, timeout=10).text, "html.parser")

    title_tag = soup.select_one(".page-title h1")
    title = title_tag.get_text(strip=True) if title_tag else ""

    # If title is empty → force price empty
    if not title:
        return {"Scraped_Title": "", "Scraped_Price": ""}

    price_tag = soup.select_one("li.item-price")
    price = price_tag.get_text(strip=True) if price_tag else ""

    return {
        "Scraped_Title": title,
        "Scraped_Price": price,
    }


In [25]:
df

,Record Id,Seller Account Name,Phone,Link,Scraped_Title,Scraped_Price
0,zcrm_6965175000001846208,Rick Leenheer,NaN,NaN,N/A,N/A
1,zcrm_6965175000002165378,Coco,+6281011094,NaN,N/A,N/A
2,zcrm_6965175000003577206,Victor Remy,NaN,https://matapropertybali.com/property/modern-4...,Modern 4-Bedroom Sanctuary in Pejeng’s Cultura...,"9,400,000,000 IDR"
3,zcrm_6965175000003577207,Cedric Henry Levi,NaN,https://matapropertybali.com/property/modern-3...,"Modern 3-Bedroom Villa in Singakerta for Sale,...","5,870,000,000 IDR"
4,zcrm_6965175000003577208,BSH,NaN,https://matapropertybali.com/property/villa-wi...,"Serene 3-Bedroom Jungle Villa in Tegalalang, Ubud","6,380,000,000 IDR"
...,...,...,...,...,...,...
527,zcrm_6965175000003616239,CP: Juli Optic,+62 831-1616-7305,https://matapropertybali.com/property/a-breath...,A breathtaking plot of land in Tegalalang for ...,"5,000,000 IDR/are /year"
528,zcrm_6965175000003616240,Balix Development,NaN,https://matapropertybali.com/property/stylish-...,"Stylish Apartments in Ubud for Sale, Bali","2,650,000,000 IDR"
529,zcrm_6965175000003616241,CP: Lanny,+62 813-1920-0900,https://matapropertybali.com/property/discover...,Stunning 3-Bedroom Villa with Ocean View in Ce...,"5,500,000,000 IDR"
530,zcrm_6965175000003616242,JSP Development,+62 812-3661-9575,https://matapropertybali.com/property/newly-bu...,Newly Built 2 bedroom Villa in Nusadua,"2,700,000,000 IDR"
